# Port Expenses API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [16]:
signal_ocean_api_key = '' #replace with your subscription key

## Port expenses API
The Port Expenses API retrieves expenses breakdown for a given port. First create connection towards
Port Expenses API as also Geo API in order to find available ports by name:

In [17]:
from signal_ocean import Connection, PortExpensesAPI
from signal_ocean.geo import GeoAPI, PortFilter
from datetime import datetime

connection = Connection(api_key=signal_ocean_api_key)
pe_api = PortExpensesAPI(connection)
geo_api = GeoAPI(connection)

#### Get the port expenses for an IMO
Now retrieve the expenses for the port of Fujairah for a given vessel IMO:

In [18]:
fujairah = geo_api.get_ports(PortFilter(name_like='Fujairah'))[0]
pe_api.get_port_expenses(imo = 9867621, port_id = fujairah.id)

PortExpenses(port_id=3153, port_canal=0, towage=0, berth=0, port_dues=40193, lighthouse=594, mooring=0, pilotage=0, quay=0, anchorage=0, agency_fees=1500, other=2277, suez_dues=0, total_cost=44565, miscellaneous_dues=0, is_estimated=False, canal_dues=0, berth_dues=0, lighthouse_dues=594, mooring_unmooring=0, quay_dues=0, anchorage_dues=0, port_agents=[])

The result is the total cost and a breakdown of the port expenses.

#### Optional port expenses calculation parameters
Depending on the port, the expenses calculation can be more precise by passing some extra parameters.
Retrieve the list of the parameters for the port of Fujairah.

In [19]:
pe_api.get_required_formula_parameters(port_id = fujairah.id)

['RoeAedToUsd', 'EstimatedTimeOfBerth', 'OperationLocationID']

Get again the expenses for the port of Fujairah, this time by also passing one of the optional
parameters retrieved above i.e. estimated time of berth:

In [20]:
pe_api.get_port_expenses(imo = 9867621, port_id = fujairah.id, estimated_time_of_berth=datetime(2018, 2, 27, 17, 48, 11))

PortExpenses(port_id=3153, port_canal=0, towage=0, berth=0, port_dues=40193, lighthouse=594, mooring=0, pilotage=0, quay=0, anchorage=0, agency_fees=1500, other=2277, suez_dues=0, total_cost=44565, miscellaneous_dues=0, is_estimated=False, canal_dues=0, berth_dues=0, lighthouse_dues=594, mooring_unmooring=0, quay_dues=0, anchorage_dues=0, port_agents=[])

#### Get the port expenses for a model vessel
Port expenses can also be calculated based on a model vessel. Use the Vessels API in order to retrieve
available vessel types:

In [21]:
from signal_ocean.vessels import VesselsAPI

vessels_api = VesselsAPI(connection)
vessel_types = vessels_api.get_vessel_types()
print(vessel_types)

(VesselType(id=-2, name='Not Set'), VesselType(id=-1, name='Unknown'), VesselType(id=6, name='LPG'), VesselType(id=3, name='Dry'), VesselType(id=4, name='Container'), VesselType(id=1, name='Tanker'), VesselType(id=5, name='LNG'))


Get expenses for Fujairah for dry vessels as of 8-3-2020:

In [22]:
dry_vessel_type = next(vt for vt in vessel_types if vt.name=='Dry')
pe_api.get_port_model_vessel_expenses(port_id = fujairah.id, vessel_type_id = dry_vessel_type.id, formula_calculation_date = datetime(2020, 3, 8, 0, 0, 0))

PortExpenses(port_id=3153, port_canal=0, towage=0, berth=0, port_dues=17425, lighthouse=0, mooring=0, pilotage=0, quay=0, anchorage=0, agency_fees=3000, other=47, suez_dues=0, total_cost=20473, miscellaneous_dues=0, is_estimated=False, canal_dues=0, berth_dues=0, lighthouse_dues=0, mooring_unmooring=0, quay_dues=0, anchorage_dues=0, port_agents=None)